In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import oscovida as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 13 April 2020

- Some plots look a little suspicous /  misleading
- lots of noise
- code base could do with refactoring
- explore that here

Get some test data

In [ ]:
import importlib
importlib.reload(cv)

In [ ]:
country = "Germany"
region, subregion = None, "LK Leer"
c, d, label = cv.get_country_data(country, region=region, subregion=subregion, verbose=True)

if False:
    c2 = c.drop(c[c.index > '2020-02-15'].index)
    d2 = d.drop(d[d.index > '2020-02-15'].index)
    # c2.loc['2020-04-01'] =  np.nan # c2.loc['2020-03-31']
    c2 = c2.dropna()
else:
    c2 = c
    d2 = d



In [ ]:
cv.overview(country=country, subregion=subregion);

In [ ]:
cv.overview(country="Germany", subregion="LK Pinneberg");

In [ ]:
cv.overview(country="New Zealand");

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
# ax.set_ylim([-50, 200])
cv.plot_daily_change(ax=ax, series=c2, color="C1");
ax = axes[1]

cv.plot_daily_change(ax=ax, series=d2, color="C0")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
cv.plot_reproduction_number(ax=ax, series=c2, labels=("LK Leer", "cases"), color_g="C1", color_R='C5');
ax = axes[1]
cv.plot_reproduction_number(ax=ax, series=d2, color_g="C0", labels=("LK Leer", "deaths"));

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
# ax.set_ylim([-50, 200])

c2, d2, _ = cv.get_country_data("Germany", subregion="LK Leer")
cv.plot_daily_change(ax=ax, series=c2, color="C1")
ax = axes[1]

cv.plot_daily_change(ax=ax, series=d2, color="C0")

In [ ]:
country = "Laos"
region, subregion = None, None
c, d, _ = cv.get_country_data(country, region=region, subregion=subregion)
c2 = c.drop(c[c.index < '2020-03-15'].index)
d2 = d.drop(d[d.index < '2020-03-15'].index)
c2.loc['2020-04-01'] =  np.nan # c2.loc['2020-03-31']
c2 = c2.dropna()

c2 = c.drop(c[c.index < '2020-03-15'].index)
d2 = d.drop(d[d.index < '2020-03-15'].index)


In [ ]:
def overview_dev(country, region=None, subregion=None, savefig=False):

    c, d = cv.get_country_data(country, region=region, subregion=subregion)

    fig, axes = plt.subplots(5, 1, figsize=(10, 12), sharex=False)
    ax = axes[0]
    plot_time_step(ax=ax, series=c, style="-C1")
    plot_time_step(ax=ax, series=d, style="-C0")

    ax = axes[1]
    plot_daily_change(ax=ax, series=c, color="C1")
    if country == "China":
        ax.set_ylim(0, 5000)

    ax = axes[2]
    plot_daily_change(ax=ax, series=d, color="C0")

    ax = axes[3]
    plot_growth_factor(ax, series=d, color="C0")
    plot_growth_factor(ax, series=c, color="C1")

    ax = axes[4]
    plot_doubling_time(ax, series=d, color="C0")
    plot_doubling_time(ax, series=c, color="C1")

    # enforce same x-axis on all plots
    for i in range(1, 5):
        axes[i].set_xlim(axes[0].get_xlim())
    for i in range(0, 5):
        axes[i].tick_params(left=True, right=True, labelleft=True, labelright=True)
        axes[i].yaxis.set_ticks_position('both')


In [ ]:
cv.overview('Laos');

In [ ]:
max([0, np.nan])

In [ ]:
#country = "Germany"
country = "China"
country = "Austria"
country = "United Kingdom"
country = "Laos"
region, subregion = None, None
c, d, _ = cv.get_country_data(country, region=region, subregion=subregion)
if False:
    c2 = c.drop(c[c.index > '2020-03-15'].index)
    d2 = d.drop(d[d.index > '2020-03-15'].index)
    # c2.loc['2020-04-01'] =  np.nan # c2.loc['2020-03-31']
    c2 = c2.dropna()
else:
    c2 = c
    d2 = d


In [ ]:
fig, ax = plt.subplots()
cv.plot_doubling_time(ax, c2, 'C0', labels=(country, "cases"))

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
cv.plot_doubling_time(ax=ax, series=c2, color="C1", minchange=0.5);
#ax.set_ylim([0, 8])

ax = axes[1]
cv.plot_doubling_time(ax=ax, series=d2, color="C0");
#ax.set_ylim([0, 100])

In [ ]:
series = c2
(f, f_label) , (f_smoothed, smoothed_label) = cv.compute_growth_factor(series)

q = series.diff().pct_change() + 1
fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
ax.plot(f.index, f.values, ".", color="C1", );
qsmooth = q.rolling(7, min_periods=1).mean()
ax.plot(qsmooth.index, qsmooth.values, "+", color="C0");
ax.set_ylim([0.8, 1.2])


ax = axes[1]
#plot_doubling_time(ax=ax, series=c2, color="C1");
dtime = cv.double_time_exponential(q, t2_minus_t1=1)
dtime_smooth = cv.double_time_exponential(qsmooth, t2_minus_t1=1)
ax.plot(dtime.index, dtime.values, 'x', label='raw data')
ax.plot(dtime_smooth.index, dtime_smooth.values, '-', label='smooth growth factor')
ax.set_ylim([0, 20])

In [ ]:
# series.pct_change?

# Playground to explore different smoothing options

In [ ]:
def compute_R_dev(daily_change, tau=4):
    """Given a time series s, estimate R based on description from RKI [1].

    [1] [Robert Koch Institute: Epidemiologisches Bulletin 17 | 2020 23. April 2020]
    https://www.rki.de/DE/Content/Infekt/EpidBull/Archiv/2020/Ausgaben/17_20.html

    Steps:

    1. Compute change from day to day
    2. Take tau-day averages (tau=4 is recommended as of April/May 2020)
    3. divide average from days 4 to 7 by averages from day 0 to 3, and use this data point for day[7]

    """
    # change = s.diff()
    change = daily_change
    mean4d = change.rolling(tau).mean()
    R = mean4d / mean4d.shift(tau)
    R2 = R.shift(-tau)  # this is not the RKI method, but seems more appropriate:
                        # we centre the reported value between the 2-intervals of length tau
                        # that have been used to compute it.
    # R2[(mean4d.shift(tau).abs() <= 0.01) & (mean4d.abs() <= 0.01)] = 1.1

    return R2

In [ ]:


fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=False)
ax = axes[0]
ax2 = axes[1]
n = 81
index = pd.date_range(start='1/1/2018', end='5/08/2018')[0:n]
x = np.linspace(-5, 15, n)
x0 = 0
sigma = 2
data = np.exp(-(x-x0)**2/sigma**2)*5 + (np.random.uniform(size=n) - 0.5) * 0.5 * 0 

s = pd.Series(data=data, index=index)

# s.iloc[n//2+2] = np.nan

# s.iloc[n//2-15] = 1

ax.bar(s.index, s.values, alpha=0.2, width=0.3)
rgauss = s.rolling(7, center=True).mean()
rmean = s.rolling(7, center=True, win_type='gaussian', min_periods=1).mean(std=2)
rgauss = s.rolling(7, center=True, win_type='gaussian', min_periods=1).mean(std=3)
ax.plot(rmean.index, rmean.values, '-', label='box')
ax.plot(rgauss.index, rgauss.values, '-', linewidth=4, alpha=0.4, label='gaussian')
R1 = compute_R_dev(s, tau=7)
ax2.plot(R1.index, R1.values, label='R')
ax2.plot(mean4d.index, mean4d.values, 'C5', label='mean4d')
ax2.plot(mean4d.index, [1] * len(mean4d), 'C6', label='1.0')

#ax2.set_ylim(top=2, bottom=0)

In [ ]:
print("s     ", s.sum())
print("rmean ", rmean.sum())
print("gaus  ", rgauss.sum())


# Plot daily change


In [ ]:
def compute_daily_change_dev(series):
    """returns (change, smooth, smooth2)

    where 'change' is a tuple of (series, label)
    and smooth is a tuple of (series, label).
    and smooth2 is a tuple of (series, label).

    'change' returns the raw data (with nan's dropped)
    'smooth' makes the data smoother
    'smooth2' does some additional smoothing - more artistic than scientific

    The 'change' under consideration, is the day-to-day change of the series.
    We assume that there is one entry per day in the Series.

    """
    diff = series.diff().dropna()
    label = ""
    change = diff, label

    # smoothed curve, technical description
    smooth_label = f"Gaussian window (stddev=3 days)"
    # shorter description
    smooth_label = f"(rolling mean)"
    rolling_series = diff.rolling(9, center=True,
                                  win_type='gaussian',
                                  min_periods=1).mean(std=3)
    smooth = rolling_series, smooth_label

    # extra smoothing for better visual effects
    rolling_series2 = rolling_series.rolling(4, center=True,
                                             win_type='gaussian',
                                             min_periods=1).mean(std=2)
    # extra smooth curve
    smooth2_label = "Smoothed " + smooth_label
    # shorter description
    smooth2_label = smooth_label

    smooth2 = rolling_series2, smooth2_label

    return change, smooth, smooth2



In [ ]:
def plot_daily_change_dev(ax, series, color, labels=None):
    """Given a series of data and matplotlib axis ax, plot the
    - difference in the series data from day to day as bars and plot a smooth
    - line to show the overall development

    - series is pandas.Series with data as index, and cumulative cases (or
    deaths)
    - color is color to be used for plotting

    See plot_time_step for documentation on other parameters.
    """

    bar_alpha = 0.2
    if labels is None:
        label = ""
        region = ""
    else:
        region, label = labels

    ax_label = region + " new " + label

    (change, change_label) , (smooth, smooth_label), \
        (smooth2, smooth2_label) = compute_daily_change_dev(series)

    ax.bar(change.index, change.values, color=color,
           label=ax_label, alpha=bar_alpha, linewidth=cv.LW)

    ax.plot(smooth2.index, smooth2.values, color=color,
            label=ax_label + " " + smooth2_label, linewidth=cv.LW)

    ax.legend()
    ax.set_ylabel('daily change')

    # labels on the right y-axis as well
    ax.tick_params(left=True, right=True, labelleft=True, labelright=True)
    ax.yaxis.set_ticks_position('both')


    # data cleaning: For France, there was a huge spike on 12 April with 26849
    # new infections. This sets the scale to be too large.
    # There was also a value of ~-2000 on 22 April. We limit the y-scale to correct
    # manually for this:
    if region == "France" and label == "cases":
        # get current limits
        ymin, ymax = ax.get_ylim()
        ax.set_ylim([max(-500, ymin), min(10000, ymax)])

    return ax



In [ ]:
fig, ax = plt.subplots()
plot_daily_change_dev(series=c2, color='C1', ax=ax)

In [ ]:
(change, change_label) , (smooth, smooth_label), \
        (smooth2, smooth2_label) = compute_daily_change_dev(c2)

In [ ]:
# data for computation or R
smooth_diff = s.diff().rolling(7,
                            center=True,
                            win_type='gaussian').mean(std=4)
R = compute_R_dev(smooth_diff)

In [ ]:
R[-20:]

In [ ]:
smooth_diff

In [ ]:
fig, ax = plt.subplots()
ax.plot(smooth_diff.index, smooth_diff.values)
# R[abs(R>10000)] = np.nan
ax.plot(R.index, R.values, label="R")
ax.legend()
#ax.set_ylim([0, 2])